In [ ]:
import sys
import os

# Add the scripts folder to the Python path
sys.path.append(os.path.abspath("../scripts"))  # Adjust the path accordingly

In [ ]:
! pip install -U nltk

In [ ]:
from load_preprocessed_data import load_ladino_pos

weak_dataset_file_path = '../data/weak/ladino-pos.txt'
weak_tags, weak_tags_dict = load_ladino_pos(weak_dataset_file_path) # import ladino tokens into custom data definition

In [ ]:
# check it worked
for item in weak_tags[0]:
    print(item)

In [ ]:
import nltk

def trainer():
    tagged_sentences = weak_tags
    formatted_sentences = [[(token.get_word(), token.get_pos()) for token in sentence] for sentence in tagged_sentences]
    words = [token.get_word() for sentence in tagged_sentences for token in sentence]
    tags = [token.get_pos() for sentence in tagged_sentences for token in sentence]
    trainer = nltk.tag.hmm.HiddenMarkovModelTrainer(states=tags, symbols=words)
    hmm_tagger = trainer.train_supervised(formatted_sentences, estimator=None)
    return hmm_tagger

hmm_tagger = trainer()